# 1. Imports

In [1]:
from get_data import init_column_map, column_map


# 2. Declarations

# 3. Subset Optimisation

In [3]:
# Initialize column_map
csv_file = './data/survey_pre_processed_data.csv'
init_column_map(csv_file)

# Build a list of columns to EXCLUDE from subset searching
exclude_cols = [
    "responseid",
    "interview_start",
    "interview_end",
    "Q10_2",        # If Q10_2 is our target
    # ...whatever else 
]

# Create  "all_variables" by including everything except the excludes
all_variables = [col for col in column_map.keys() if col not in exclude_cols]

print("Candidate columns:", all_variables)

Candidate columns: ['\ufeffresponseid', 'gender', 'colour_plus', 'region', 'list', 'Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_99', 'Q2', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q3_5', 'Q4', 'Q4_98_other', 'Q4b_1', 'Q4b_2', 'Q4b_3', 'Q4b_4', 'Q4b_98', 'Q4b_98_recoded', 'Q4b_98_other', 'Q4b_5', 'Q4b_6', 'Q4b_7', 'Q4b_8', 'Q4b_9', 'Q4b_10', 'Q4b_99', 'Q5_1', 'Q5_2', 'Q5_3', 'Q5_4', 'Q5_99', 'Q6ax1_1', 'Q6ax1_2', 'Q6ax1_3', 'Q6ax2_1', 'Q6ax2_2', 'Q6ax2_3', 'Q6ax3_1', 'Q6ax3_2', 'Q6ax3_3', 'Q6ax4_1', 'Q6ax4_2', 'Q6ax4_3', 'Q6bx1_1', 'Q6bx1_2', 'Q6bx1_3', 'Q6bx2_1', 'Q6bx2_2', 'Q6bx2_3', 'Q6bx3_1', 'Q6bx3_2', 'Q6bx3_3', 'Q6bx4_1', 'Q6bx4_2', 'Q6bx4_3', 'Q7_1', 'Q7_2', 'Q7_3', 'Q8_1', 'Q8_2', 'Q8_99', 'Q9', 'Q10_1', 'oQ11', 'Q13', 'Q13_98_other', 'Q13_1', 'Q13_2', 'Q13_3', 'Q14_1', 'Q14_2', 'Q14_3', 'Q14_4', 'Q14_5', 'Q14_6', 'Q14_7', 'Q14_8', 'Q14_98', 'Q14_98_other', 'Q14_99', 'Q15_1', 'Q15_2', 'Q15_3', 'Q15_4', 'Q15_5', 'Q15_6', 'Q15_7', 'Q15_8', 'Q15_9', 'Q15_10', 'Q15_98', 'Q15_98_other', 'Q15_99', 'Q

In [4]:
target_variable = 'Q10_2'  # "I would be interested in installing V2G..."

In [5]:
import itertools
from build_model import build_v2g_model_multinomial  
from sklearn.model_selection import cross_val_score

def evaluate_subset(csvfile, var_subset, target="Q10_2"):
    """
    Builds and cross-validates a logistic regression model 
    on the given subset of variables. Returns mean CV accuracy.
    """
    model, X, y, _, _ = build_v2g_model_multinomial(
        csvfile=csvfile,
        input_variables=list(var_subset),
        target_variable=target,
        do_normalize=True,
        test_split_ratio=0.0
    )
    # Evaluate with 5-fold cross-validation
    scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")
    return scores.mean()

best_acc = 0
best_combo = None

# Let’s do an example that only tries subsets up to size 3 for demonstration:
max_subset_size = 3

for r in range(1, max_subset_size + 1):
    for subset in itertools.combinations(all_variables, r):
        acc = evaluate_subset(csv_file, subset, target=target_variable)
        if acc > best_acc:
            best_acc = acc
            best_combo = subset

print("Best subset:", best_combo)
print("Best subset accuracy:", best_acc)


Training accuracy: 0.3446244477172312


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.